In [1]:
import os
import requests
import json

In [9]:
WB_TOKEN = os.getenv('WB_TOKEN')


def upload_single_image_to_wb(file_path, nm_id, api_key, photo_number=1):
  
    url = "https://content-api.wildberries.ru/content/v3/media/file"
    
    headers = {
        "Authorization": api_key,
        "X-Nm-Id": str(nm_id),
        "X-Photo-Number": str(photo_number)
    }
    
    with open(file_path, "rb") as f:
        files = {
            "uploadfile": (os.path.basename(file_path), f, "image/jpeg")
        }
        
        try:
            response = requests.post(url, headers=headers, files=files)
        except requests.RequestException as e:
            print(f"Ошибка при отправке запроса: {e}")
            return
        
        if response.status_code == 200:
            resp_json = response.json()
            if not resp_json.get("error", False):
                print(f"Успешно загружено изображение '{os.path.basename(file_path)}' на nmID={nm_id}.")
            else:
                print(f"Ошибка Wildberries при загрузке: {resp_json.get('errorText', 'Неизвестная ошибка')}")
        else:
            print(f"Ошибка ответа сервера. Код: {response.status_code}, файл: {os.path.basename(file_path)}")

def process_vendor_list(vendor_list, images_folder, api_key):
    """
    Проходит по списку vendor_list, проверяет, существует ли соответствующий файл в images_folder,
    и при наличии вызывает функцию загрузки на WB.
    
    :param vendor_list: Список словарей вида [{'nmID': ..., 'vendorCode': ' zip_lock_XXXX_YY'}, ...]
    :param images_folder: Путь к папке, где лежат файлы (например, 'images_900_1200').
    :param api_key: API-ключ для Wildberries.
    """
    for item in vendor_list:
        nm_id = item["nmID"]
        vendor_code = item["vendorCode"]
        
        # Ожидаем формат vendorCode = "zip_lock_9604677_30"
        # Удаляем " zip_lock_" в начале:
        # Неподходящий формат vendorCode: 'zip_lock_9604896_100'. Пропускаем.
        
        if vendor_code.startswith("zip_"):
        
            code_part = vendor_code[4:]  # "9604677_30"
            # Разделяем по символу "_"
            parts = code_part.split("_", maxsplit=1)
            if len(parts) == 2:
                product_id, quantity_str = parts
                # Формируем имя файла: "9604677-30.jpg"
                image_name = f"{product_id}-{quantity_str}.jpg"
                file_path = os.path.join(images_folder, image_name)
                
                # Проверяем, существует ли файл
                if os.path.isfile(file_path):
                    print(f"Файл '{image_name}' найден. Загружаем на nmID={nm_id}...")
                    upload_single_image_to_wb(file_path, nm_id, api_key, photo_number=1)
                else:
                    print(f"Файл '{image_name}' не найден в папке '{images_folder}'. Пропускаем.")
            else:
                print(f"Неподходящий формат vendorCode: '{vendor_code}'. Пропускаем.")
        else:
            print(f"Неподходящий формат vendorCode: '{vendor_code}'. Пропускаем.")


In [10]:
all_cards_list = "https://content-api.wildberries.ru/content/v2/get/cards/list"
headers = {
    "Authorization": WB_TOKEN,
}
body =             {
          "settings": {                      
            "cursor": {
              "limit": 100
            },
            "filter": {
              "withPhoto": 0
            }
          }
        }


resp = requests.post(all_cards_list, headers=headers, json=body)

In [11]:
d = json.loads(resp.text)
d

{'cards': [{'nmID': 324571936,
   'imtID': 305916447,
   'nmUUID': '0194c713-e498-7c72-9847-a53a167d3230',
   'subjectID': 3756,
   'subjectName': 'Пакеты упаковочные',
   'vendorCode': 'zip_9604945_100',
   'brand': '',
   'title': 'Пакет EVA с замком слайдер 30*40 см 60 мкм матовый 50 шт.',
   'description': 'Матовый пакет-слайдер шириной 300 мм, высотой 400 мм и толщиной 60 мкм, с вентиляционными отверстиями. Изготовлен из EVA — плёнки высокой прочности. Стоимость указана за 50 штук.\nПрименяется для упаковки и хранения:\nодежды и обуви малых размеров (соответствует требованиям маркетплейсов);\nфруктов и овощей;\nбытовой химии;\nдокументов;\nмедикаментов и косметических средств;\nсредств личной гигиены.\nПреимущества:\nмногоразовый;\nпрочность;\nпростой в использовании замок;\nпрезентабельный вид пакета;\nнизкая цена.',
   'needKiz': False,
   'dimensions': {'width': 40, 'height': 2, 'length': 30, 'isValid': True},
   'sizes': [{'chrtID': 487476625,
     'techSize': '0',
     'wbSiz

In [12]:
newNmIdVendorCodes = [{"nmID": row["nmID"], "vendorCode": row["vendorCode"]} for row in d["cards"]]

In [13]:
newNmIdVendorCodes

[{'nmID': 324571936, 'vendorCode': 'zip_9604945_100'},
 {'nmID': 324571935, 'vendorCode': 'zip_9604595_100'},
 {'nmID': 324571934, 'vendorCode': 'zip_9604946_100'},
 {'nmID': 324571933, 'vendorCode': 'zip_9604947_100'},
 {'nmID': 324571932, 'vendorCode': 'zip_9604948_100'},
 {'nmID': 324571931, 'vendorCode': 'zip_9604943_100'},
 {'nmID': 324571930, 'vendorCode': 'zip_9604942_100'},
 {'nmID': 324571929, 'vendorCode': 'zip_9604632_100'},
 {'nmID': 324571928, 'vendorCode': 'zip_9604944_100'},
 {'nmID': 324571927, 'vendorCode': 'zip_9604591_100'},
 {'nmID': 324571926, 'vendorCode': 'zip_9604590_100'}]

In [14]:
process_vendor_list(newNmIdVendorCodes, "./zip_images", WB_TOKEN)

Файл '9604945-100.jpg' найден. Загружаем на nmID=324571936...
Успешно загружено изображение '9604945-100.jpg' на nmID=324571936.
Файл '9604595-100.jpg' найден. Загружаем на nmID=324571935...
Успешно загружено изображение '9604595-100.jpg' на nmID=324571935.
Файл '9604946-100.jpg' найден. Загружаем на nmID=324571934...
Успешно загружено изображение '9604946-100.jpg' на nmID=324571934.
Файл '9604947-100.jpg' найден. Загружаем на nmID=324571933...
Успешно загружено изображение '9604947-100.jpg' на nmID=324571933.
Файл '9604948-100.jpg' найден. Загружаем на nmID=324571932...
Успешно загружено изображение '9604948-100.jpg' на nmID=324571932.
Файл '9604943-100.jpg' найден. Загружаем на nmID=324571931...
Успешно загружено изображение '9604943-100.jpg' на nmID=324571931.
Файл '9604942-100.jpg' найден. Загружаем на nmID=324571930...
Успешно загружено изображение '9604942-100.jpg' на nmID=324571930.
Файл '9604632-100.jpg' найден. Загружаем на nmID=324571929...
Успешно загружено изображение '9604